# Homework 2: Stanford Sentiment Treebank

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2019"

## Contents

1. [Overview](#Overview)
1. [Methodological note](#Methodological-note)
1. [Set-up](#Set-up)
1. [A softmax baseline](#A-softmax-baseline)
1. [RNNClassifier wrapper](#RNNClassifier-wrapper)
1. [Error analysis](#Error-analysis)
1. [Homework questions](#Homework-questions)
  1. [Sentiment words alone [2 points]](#Sentiment-words-alone-[2-points])
  1. [A more powerful vector-summing baseline [3 points]](#A-more-powerful-vector-summing-baseline-[3-points])
  1. [Your original system [4 points]](#Your-original-system-[4-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the Stanford Sentiment Treebank (SST). The homework questions ask you to implement some baseline systems, and the bake-off challenge is to define a system that does extremely well at the SST task.

We'll focus on the ternary task as defined by `sst.ternary_class_func`.

The SST test set will be used for the bake-off evaluation. This dataset is already publicly distributed, so we are counting on people not to cheat by develping their models on the test set. You must do all your development without using the test set at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. __Much of the scientific integrity of our field depends on people adhering to this honor code__. 

Our only additional restriction is that __you cannot make any use of the subtree labels__. This corresponds to the 'Root' condition in the paper. As we discussed in class, the subtree labels are a really interesting feature of SST, but bringing them in results in a substantially different learning problem.

One of our goals for this homework and bake-off is to encourage you to engage in __the basic development cycle for supervised models__, in which you

1. Write a new feature function. We recommend starting with something simple.
1. Use `sst.experiment` to evaluate your new feature function, with at least `fit_softmax_classifier`.
1. If you have time, compare your feature function with `unigrams_phi` using `sst.compare_models` or `sst.compare_models_mcnemar`. (For discussion, see [this notebook section](sst_02_hand_built_features.ipynb#Statistical-comparison-of-classifier-models).)
1. Return to step 1, or stop the cycle and conduct a more rigorous evaluation with hyperparameter tuning and assessment on the `dev` set.

[Error analysis](#Error-analysis) is one of the most important methods for steadily improving a system, as it facilitates a kind of human-powered hill-climbing on your ultimate objective. Often, it takes a careful human analyst just a few examples to spot a major pattern that can lead to a beneficial change to the feature representations.

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). However, if you don't use `sst.experiment` as below, then make sure you're training only on `train`, evaluating on `dev`, and that you report with 

```
from sklearn.metrics import classification_report
classification_report(y_dev, predictions)
```
where `y_dev = [y for tree, y in sst.dev_reader(class_func=sst.ternary_class_func)]`. We'll focus on the value at `macro avg` under `f1-score` in these reports.

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [3]:
from bert_serving.client import BertClient 
from collections import Counter
import copy
from functools import partial
import numpy as np
import os
import pandas as pd
import random
import scipy.stats
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import sst
import sys
import torch.nn as nn
from torch_rnn_classifier import TorchRNNClassifier
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_tree_nn import TorchTreeNN
import utils

In [8]:
DATE_HOME = '/home/kd/data/data'

GLOVE_HOME = os.path.join(DATE_HOME, 'glove.6B')

VSMDATA_HOME = os.path.join(DATE_HOME, 'vsmdata')

SST_HOME = os.path.join(DATE_HOME, 'trees')

## A softmax baseline

This example is here mainly as a reminder of how to use our experimental framework with linear models.

In [5]:
def unigrams_phi(tree):
    """The basis for a unigrams feature function.
    
    Parameters
    ----------
    tree : nltk.tree
        The tree to represent.
    
    Returns
    -------    
    Counter
        A map from strings to their counts in `tree`. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    
    """
    return Counter(tree.leaves())

Thin wrapper around `LogisticRegression` for the sake of `sst.experiment`:

In [6]:
def fit_softmax_classifier(X, y):        
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')
    mod.fit(X, y)
    return mod

The experimental run with some notes:

In [9]:
softmax_experiment = sst.experiment(
    SST_HOME,
    unigrams_phi,                      # Free to write your own!
    fit_softmax_classifier,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func) # Fixed by the bake-off rules.

              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101



`softmax_experiment` contains a lot of information that you can use for analysis; see [this section below](#Error-analysis) for starter code.

## RNNClassifier wrapper

This section illustrates how to use `sst.experiment` with RNN and TreeNN models.

To featurize examples for an RNN, we just get the words in order, letting the model take care of mapping them into an embedding space.

In [10]:
def rnn_phi(tree):
    return tree.leaves()    

In [11]:
X_rnn_train, y_rnn_train = sst.build_rnn_dataset(
    SST_HOME, sst.train_reader, class_func=sst.ternary_class_func)

In [12]:
sst_train_vocab = utils.get_vocab(X_rnn_train, n_words=10000)

In [13]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [14]:
glove_embedding, sst_glove_vocab = utils.create_pretrained_embedding(
    glove_lookup, sst_train_vocab)

The model wrapper gets the vocabulary using `sst.get_vocab`. If you want to use pretrained word representations in here, then you can have `fit_rnn_classifier` build that space too; see [this notebook section for details](sst_03_neural_networks.ipynb#Pretrained-embeddings).

In [15]:
def fit_rnn_classifier(X, y):    
    sst_glove_vocab = utils.get_vocab(X, n_words=10000)     
    mod = TorchRNNClassifier(
        sst_glove_vocab, 
        eta=0.05,
        embedding=None,
        batch_size=1000,
        embed_dim=50,
        hidden_dim=50,
        max_iter=50,
        l2_strength=0.001,
        bidirectional=True,
        hidden_activation=nn.ReLU())
    mod.fit(X, y)
    return mod

In [16]:
rnn_experiment = sst.experiment(
    SST_HOME,
    rnn_phi,
    fit_rnn_classifier, 
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_reader=sst.dev_reader)

Finished epoch 50 of 50; error is 2.3360441029071814

              precision    recall  f1-score   support

    negative      0.578     0.736     0.647       428
     neutral      0.319     0.188     0.236       229
    positive      0.667     0.633     0.650       444

    accuracy                          0.580      1101
   macro avg      0.521     0.519     0.511      1101
weighted avg      0.560     0.580     0.563      1101



In [17]:
def find_errors(experiment):
    """Find mistaken predictions.
    
    Parameters
    ----------
    experiment : dict
        As returned by `sst.experiment`.
        
    Returns
    -------
    pd.DataFrame
    
    """
    raw_examples = experiment['assess_dataset']['raw_examples']
    raw_examples = [" ".join(tree.leaves()) for tree in raw_examples]
    df = pd.DataFrame({
        'raw_examples': raw_examples,
        'predicted': experiment['predictions'],
        'gold': experiment['assess_dataset']['y']})
    df['correct'] = df['predicted'] == df['gold']
    return df

In [18]:
softmax_analysis = find_errors(softmax_experiment)

In [19]:
rnn_analysis = find_errors(rnn_experiment)

Here we merge the sotmax and RNN experiments into a single DataFrame:

In [20]:
analysis = softmax_analysis.merge(
    rnn_analysis, left_on='raw_examples', right_on='raw_examples')

analysis = analysis.drop('gold_y', axis=1).rename(columns={'gold_x': 'gold'})

The following code collects a specific subset of examples; small modifications to its structure will give you different interesting subsets:

In [21]:
# Examples where the softmax model is correct, the RNN is not,
# and the gold label is 'positive'

error_group = analysis[
    (analysis['predicted_x'] == analysis['gold'])
    &
    (analysis['predicted_y'] != analysis['gold'])    
    &
    (analysis['gold'] == 'positive')
]

In [22]:
error_group.shape[0]

75

In [23]:
for ex in error_group['raw_examples'].sample(5):
    print("="*70)
    print(ex)

Bogdanovich tantalizes by offering a peep show into the lives of the era 's creme de la celluloid .
For the first time in years , De Niro digs deep emotionally , perhaps because he 's been stirred by the powerful work of his co-stars .
It haunts you , you ca n't forget it , you admire its conception and are able to resolve some of the confusions you had while watching it .
What 's surprising about Full Frontal is that despite its overt self-awareness , parts of the movie still manage to break past the artifice and thoroughly engage you .
Lovely and poignant .


## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Sentiment words alone [2 points]

NLTK includes an easy interface to [Minqing Hu and Bing Liu's __Opinion Lexicon__](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), which consists of a list of positive words and a list of negative words. How much of the ternary SST story does this lexicon tell?

For this problem, submit code to do the following:

1. Create a feature function `op_unigrams` on the model of `unigrams_phi` above, but filtering the vocabulary to just items that are members of the Opinion Lexicon. Submit this feature function.

1. Evaluate your feature function with `sst.experiment`, with all the same parameters as were used to create `softmax_experiment` in [A softmax baseline](#A-softmax-baseline) above, except of course for the feature function.

1. Use `utils.mcnemar` to compare your feature function with the results in `softmax_experiment`. The information you need for this is in `softmax_experiment` and your own `sst.experiment` results. Submit your evaluation code. You can assume `softmax_experiment` is already in memory, but your code should create the other objects necessary for this comparison.

In [25]:
from nltk.corpus import opinion_lexicon

# Use set for fast membership checking:
positive = set(opinion_lexicon.positive())
negative = set(opinion_lexicon.negative())
words = set(opinion_lexicon.words())


In [26]:
def op_unigrams_words(tree, words):
    leaves = Counter(copy.deepcopy(tree.leaves()))
    for key in list(leaves):
        if key in words:
            del leaves[key]

    return leaves

op_unigrams = partial(op_unigrams_words, words=words)

In [27]:
softmax_experiment = sst.experiment(
    SST_HOME,
    unigrams_phi,                      # Free to write your own!
    fit_softmax_classifier,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func) # Fixed by the bake-off rules.

              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101



In [28]:
op_experiment = sst.experiment(
    SST_HOME,
    op_unigrams,                      # Free to write your own!
    fit_softmax_classifier,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func) # Fixed by the bake-off rules.

              precision    recall  f1-score   support

    negative      0.575     0.607     0.591       428
     neutral      0.266     0.109     0.155       229
    positive      0.580     0.725     0.645       444

    accuracy                          0.551      1101
   macro avg      0.474     0.481     0.463      1101
weighted avg      0.513     0.551     0.522      1101



In [29]:
_ = sst.compare_models(
    SST_HOME,
    unigrams_phi,
    fit_softmax_classifier,
    stats_test=scipy.stats.wilcoxon,
    trials=10,
    phi2=op_unigrams,  # Defaults to same as first required argument.
    train_func2=fit_softmax_classifier, # Defaults to same as second required argument.
    reader=sst.train_reader, 
    train_size=0.7, 
    class_func=sst.ternary_class_func, 
    score_func=utils.safe_macro_f1)

Model 1 mean: 0.513
Model 2 mean: 0.474
p = 0.005


In [30]:
m = utils.mcnemar(
    softmax_experiment['assess_dataset']['y'], 
    op_experiment['predictions'],
    softmax_experiment['predictions'])

p = "p < 0.0001" if m[1] < 0.0001 else m[1]

print("McNemar's test: {0:0.02f} ({1:})".format(m[0], p))

McNemar's test: 20.17 (p < 0.0001)


### A more powerful vector-summing baseline [3 points]

In [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the ternary SST problem in which each example is modeled as the sum of its 50-dimensional GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better with these representations, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. 

To address this question, rerun the experiment with `TorchShallowNeuralClassifier` as the classifier. Specs:
* Use `sst.experiment` to conduct the experiment. 
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `nn.Tanh` or `nn.ReLU`.
* (For all other parameters to `TorchShallowNeuralClassifier`, use the defaults.)

For this problem, submit code to do the following:

1. Your model wrapper function around `TorchShallowNeuralClassifier`. This function should implement the requisite cross-validation; see [this notebook section](sst_02_hand_built_features.ipynb#Hyperparameter-search) for examples.
1. The classification report as printed by `sst.experiment`. (This will print out when you run `sst.experiment`. That print-out suffices.)
2. The optimal hyperparameters chosen in your experiment. (This too will print out when you run `sst.experiment`. The print-out again suffices.)

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be variation in the results.) However, the primary goal of this question is to get you thinking more about this strikingly good baseline feature representation scheme for SST, so we're sort of hoping you feel compelled to try out variations on your own.

In [31]:
def fit_nn_classifier(X, y):
    mod = TorchShallowNeuralClassifier(
        hidden_dim=50, max_iter=100)
    mod.fit(X, y)
    return mod

In [32]:
def fit_shallow_nn_classifier_with_crossvalidation(X, y):
    basemod = fit_nn_classifier(X, y)
    cv = 3
    param_grid = {
                    'hidden_dim': [50, 100, 200], 
                    'hidden_activation': [nn.Tanh(), nn.ReLU()]
                 }    
    best_mod = utils.fit_classifier_with_crossvalidation(
        X, y, basemod, cv, param_grid)
    return best_mod

In [33]:
shallow_nn_classifier_experiment = sst.experiment(
    SST_HOME,
    unigrams_phi,
    fit_shallow_nn_classifier_with_crossvalidation, 
    class_func=sst.ternary_class_func)

Finished epoch 100 of 100; error is 0.00025416948483325542

Best params: {'hidden_activation': Tanh(), 'hidden_dim': 200}
Best score: 0.505
              precision    recall  f1-score   support

    negative      0.615     0.571     0.592      1016
     neutral      0.235     0.230     0.233       457
    positive      0.634     0.683     0.658      1091

    accuracy                          0.558      2564
   macro avg      0.495     0.494     0.494      2564
weighted avg      0.555     0.558     0.556      2564



### Your original system [4 points]

Your task is to develop an original model for the SST ternary problem, using only the root-level labels (again, __you cannot make any use of the subtree labels__). There are many options. If you spend more than a few hours on this homework problem, you should consider letting it grow into your final project! Here are some relatively manageable ideas that you might try:

1. We didn't systematically evaluate the `bidirectional` option to the `TorchRNNClassifier`. Similarly, that model could be tweaked to allow multiple LSTM layers (at present there is only one), and you could try adding layers to the classifier portion of the model as well.

1. We've already glimpsed the power of rich initial word representations, and later in the course we'll see that smart initialization usually leads to a performance gain in NLP, so you could perhaps achieve a winning entry with a simple model that starts in a great place.

1. The [practical introduction to contextual word representations](contextualreps.ipynb) (to be discussed later in the quarter) covers pretrained representations and interfaces that are likely to boost the performance of any system.

1. The `TreeNN` and `TorchTreeNN` don't perform all that well, and this could be for the same reason that RNNs don't peform well: the gradient signal doesn't propagate reliably down inside very deep trees. [Tai et al. 2015](https://aclanthology.info/papers/P15-1150/p15-1150) sought to address this with TreeLSTMs, which are fairly easy to implement in PyTorch.

1. In the [distributed representations as features](#Distributed-representations-as-features) section, we just summed  all of the leaf-node GloVe vectors to obtain a fixed-dimensional representation for all sentences. This ignores all of the tree structure. See if you can do better by paying attention to the binary tree structure: write a function `glove_subtree_phi` that obtains a vector representation for each subtree by combining the vectors of its daughters, with the leaf nodes again given by GloVe (any dimension you like) and the full representation of the sentence given by the final vector obtained by this recursive process. You can decide on how you combine the vectors. 

1. If you have a lot of computing resources, then you can fire off a large hyperparameter search over many parameter values. All the model classes for this course are compatible with the `scikit-learn` and [scikit-optimize](https://scikit-optimize.github.io) methods, because they define the required functions for getting and setting parameters.

We want to emphasize that this needs to be an __original__ system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it.

__Please include a brief prose description of your system along with your code, to help the teaching team understand the structure of your system.__

In [84]:
bc = BertClient(check_length=False)

In [85]:
sst_train_reader = sst.train_reader(
    SST_HOME, class_func=sst.ternary_class_func)

sst_train = [(" ".join(t.leaves()), label) for t, label in sst_train_reader]

In [86]:
sst_dev_reader = sst.dev_reader(
    SST_HOME, class_func=sst.ternary_class_func)

sst_dev = [(" ".join(t.leaves()), label) for t, label in sst_dev_reader]

In [145]:
X_str_train, y_train = zip(*sst_train)

In [148]:
def rebalance_classes(X, Y):
    X_train_list = [x for x in X]
    y_train_list = [y for y in Y]
    total_train = len(Y)
    # This is very hacky, lazy way to upsample
    # Proper way is to sample with replacement N_desired times
    indices_train_neutral = [i for i in range(total_train) if y_train_list[i] == 'neutral']
    x_neutral = [X_train_list[i] for i in indices_train_neutral]
    y_neutral = [y_train_list[i] for i in indices_train_neutral]
    x_train_balanced = tuple(X_train_list + x_neutral)
    y_train_balanced = tuple(y_train_list + y_neutral)
    return x_train_balanced, y_train_balanced

In [149]:
X_str_train, y_train = rebalance_classes(X_str_train, y_train)

In [158]:
X_str_dev, y_dev = zip(*sst_dev)

In [154]:
X_bert_train, bert_train_toks = bc.encode(
    list(X_str_train), show_tokens=True)

In [92]:
X_bert_dev, bert_dev_toks = bc.encode(
    list(X_str_dev), show_tokens=True)

In [93]:
def bert_reduce_mean(X):
    return X.mean(axis=1)  

In [94]:
X_bert_train_mean = bert_reduce_mean(X_bert_train)

In [95]:
X_bert_train_mean.shape[1]

768

In [101]:
bert_lookup = {}

for (sents, reps) in ((X_str_train, X_bert_train_mean), 
                      (X_str_dev, X_bert_dev_mean)):
    assert len(sents) == len(reps)
    for s, rep in zip(sents, reps):
        bert_lookup[s] = rep

In [102]:
def bert_sentence_phi(tree):
    s = " ".join(tree.leaves())
    return bert_lookup[s]

In [103]:
def fit_wide_shallow_network(X, y):
    mod = TorchShallowNeuralClassifier(
        max_iter=100, hidden_dim=300)
    mod.fit(X, y)
    return mod

In [104]:
%%time 
_ = sst.experiment(
    SST_HOME,
    bert_sentence_phi,
    fit_wide_shallow_network,
    train_reader=sst.train_reader, 
    assess_reader=sst.dev_reader, 
    class_func=sst.ternary_class_func,
    vectorize=False)

Finished epoch 100 of 100; error is 0.08013642579317093

              precision    recall  f1-score   support

    negative      0.760     0.738     0.749       428
     neutral      0.394     0.389     0.391       229
    positive      0.780     0.806     0.793       444

    accuracy                          0.693      1101
   macro avg      0.644     0.644     0.644      1101
weighted avg      0.692     0.693     0.692      1101

CPU times: user 1min 35s, sys: 1min 15s, total: 2min 51s
Wall time: 32.3 s


In [109]:
bert_rnn_layers = TorchRNNClassifier(
    vocab=[],
    max_iter=70,
    num_layers=3,
    use_embedding=False)

In [110]:
%time _ = bert_rnn_layers.fit(X_bert_train, y_train)

Finished epoch 70 of 70; error is 0.29796700179576874

CPU times: user 19min 11s, sys: 11min 2s, total: 30min 13s
Wall time: 5min 4s


In [111]:
bert_rnn_preds_layers = bert_rnn_layers.predict(X_bert_dev)

In [112]:
print(classification_report(y_dev, bert_rnn_preds_layers, digits=3))

              precision    recall  f1-score   support

    negative      0.759     0.708     0.733       428
     neutral      0.345     0.358     0.351       229
    positive      0.763     0.797     0.780       444

    accuracy                          0.671      1101
   macro avg      0.622     0.621     0.621      1101
weighted avg      0.675     0.671     0.672      1101



In [117]:
def fit_rnn_classifier_bert(X, y):    
    mod = TorchRNNClassifier(
        vocab=[],
        max_iter=80,
        use_embedding=False)
    mod.fit(X, y)
    return mod

In [118]:
def fit_shallow_nn_classifier_with_crossvalidation(X, y):
    basemod = fit_rnn_classifier_bert(X, y)
    cv = 3
    param_grid = {
                    'hidden_dim': [50, 100, 200], 
                    'hidden_activation': [nn.Tanh(), nn.ReLU()],
                    'layers' : [1,2,3,4,8]
                 }    
    best_mod = utils.fit_classifier_with_crossvalidation(
        X, y, basemod, cv, param_grid)
    return best_mod

In [120]:
%time _ = fit_shallow_nn_classifier_with_crossvalidation(X_bert_train, y_train)

Finished epoch 80 of 80; error is 0.3117343001067638435

Best params: {'hidden_activation': ReLU(), 'hidden_dim': 50, 'layers': 3}
Best score: 0.631
CPU times: user 1d 13h 34min 18s, sys: 20h 53min 22s, total: 2d 10h 27min 40s
Wall time: 9h 49min 31s


In [174]:
def fit_rnn_classifier_bert_tuned(X, y):    
    mod = TorchRNNClassifier(
        vocab=[],
        eta=0.05,
        hidden_dim=50,
        max_iter=80,
        l2_strength=0.001,
        num_layers=3,
        use_embedding=False,
        hidden_activation=nn.ReLU())
    mod.fit(X, y)
    return mod

In [175]:
%time trained_tuned_bert = fit_rnn_classifier_bert_tuned(X_bert_train, y_train)

Finished epoch 80 of 80; error is 5.8199448585510256

CPU times: user 23min 8s, sys: 14min 37s, total: 37min 45s
Wall time: 6min 20s


In [176]:
bert_rnn_preds_layers = trained_tuned_bert.predict(X_bert_dev)

In [177]:
print(classification_report(y_dev, bert_rnn_preds_layers, digits=3))

              precision    recall  f1-score   support

    negative      0.798     0.657     0.721       428
     neutral      0.383     0.520     0.441       229
    positive      0.799     0.788     0.794       444

    accuracy                          0.681      1101
   macro avg      0.660     0.655     0.652      1101
weighted avg      0.712     0.681     0.692      1101



## Bake-off [1 point]

The bake-off will begin on April 22. The announcement will go out on Piazza. As we said above, the bake-off evaluation data is the official SST test set release. For this bake-off, you'll evaluate your original system from the above homework problem on the test set, using the ternary class problem. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.
1. As noted above, __you cannot make any use of the subtree labels__.

To enter the bake-off, upload this notebook on Canvas:

https://canvas.stanford.edu/courses/99711/assignments/187246

The cells below this one constitute your bake-off entry.

Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

The bake-off will close at 4:30 pm on April 24. Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

In [183]:
# NOTE: I re-ran test (3) times, because I forgot to re-weight classes, and use proper hyperparameter values 
# from fit_rnn_classifier(). I decided to fine-tune BERT, since I thought it would be good learning exercise.
# I am *not* enrolled in this class, and I realize using test more than once is verboten, in general.
# I don't intend to do so, in future. This was purely for didactic purposes.

In [179]:
X_str_test, y_test = zip(*sst_test)

In [180]:
X_bert_test, bert_test_toks = bc.encode(
    list(X_str_test), show_tokens=True)

In [181]:
bert_rnn_preds_layers_result = trained_tuned_bert.predict(X_bert_test)

In [182]:
print(classification_report(y_test, bert_rnn_preds_layers_result, digits=3))

              precision    recall  f1-score   support

    negative      0.831     0.652     0.731       912
     neutral      0.323     0.524     0.400       389
    positive      0.855     0.811     0.832       909

    accuracy                          0.695      2210
   macro avg      0.670     0.663     0.654      2210
weighted avg      0.751     0.695     0.714      2210

